In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import statsmodels.api as sm
import glob
from functools import reduce
import numpy as np
from scipy import stats

In [ ]:
# Load the dataset from severals files.

#Home PC data classpath

surveyV1_df = pd.read_csv("D:/ExportFiles/TVSEP2022SurveyV1.csv", low_memory=False)
shocks_df = pd.read_csv("D:/ExportFiles/shocks.csv",low_memory=False)
shocks_details_df = pd.read_csv("D:/ExportFiles/shocks_detail.csv",low_memory=False)

#BSRU PC classpath
"""
surveyV1_df = pd.read_csv("E:/ExportFiles/TVSEP2022SurveyV1.csv", low_memory=False)
shocks_df = pd.read_csv("E:/ExportFiles/shocks.csv",low_memory=False)
shocks_details_df = pd.read_csv("E:/ExportFiles/shocks_detail.csv",low_memory=False)
"""

surveyV1Col = list(surveyV1_df.columns)
shocksCol = list(shocks_df.columns)
shocksDeCol = list(shocks_details_df.columns)



#data = data[shockRelated]

FileNotFoundError: [Errno 2] No such file or directory: 'D:/ExportFile/TVSEP2022SurveyV1.csv'

In [7]:
shocks_details_df.head()

,Unnamed: 0,interview__key,interview__id,shocks__id,shocks_detail__id,v31102d,v31102a__0,v31102a__1,v31102a__2,v31102a__3,...,shockdate,corryear,v31127__0,v31127__1,v31127__2,v31127__3,v31127__4,v31127__5,v31127__98,specify_v31127
0,0,61-84-00-54,e4caefaf3c074773a6e478861d18f0a7,2,0,Yes,NaN,NaN,NaN,NaN,...,2022-01-01T00:00:00,2022.0,0,0,0,0,1,0,0,NaN
1,1,61-84-00-54,e4caefaf3c074773a6e478861d18f0a7,3,0,Yes,NaN,NaN,NaN,NaN,...,2020-01-01T00:00:00,2020.0,1,0,0,0,0,0,0,NaN
2,2,59-46-95-71,419de10d4606492390aaec2a7fb1e5b9,10,0,Yes,NaN,NaN,NaN,NaN,...,2020-08-01T00:00:00,2020.0,0,0,0,1,1,0,0,NaN
3,3,16-60-25-95,22b3b9365e3246b3821baad080732b6b,62,0,Yes,NaN,NaN,NaN,NaN,...,2021-07-01T00:00:00,2021.0,1,0,0,0,0,0,0,NaN
4,4,36-89-97-65,782b881d3751484d8cbdf49f3daae2ac,1,0,Yes,NaN,NaN,NaN,NaN,...,2021-04-01T00:00:00,2021.0,1,0,0,0,0,0,0,NaN


In [8]:
descriptive_stats = shocks_details_df['v31102d'].describe()
print(descriptive_stats)

count     2177
unique       2
top        Yes
freq      1334
Name: v31102d, dtype: object


In [9]:
# Display basic information about the dataset
print("\nDataset Info:")
print(shocks_df.info())

# Display the first few rows of the dataset
print("\nFirst few rows:")
print(shocks_df.head())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2177 entries, 0 to 2176
Columns: 101 entries, Unnamed: 0 to specify_v31127
dtypes: float64(75), int64(10), object(16)
memory usage: 1.7+ MB
None

First few rows:
   Unnamed: 0 interview__key                     interview__id  shocks__id  \
0           0    61-84-00-54  e4caefaf3c074773a6e478861d18f0a7           2   
1           1    61-84-00-54  e4caefaf3c074773a6e478861d18f0a7           3   
2           2    59-46-95-71  419de10d4606492390aaec2a7fb1e5b9          10   
3           3    16-60-25-95  22b3b9365e3246b3821baad080732b6b          62   
4           4    36-89-97-65  782b881d3751484d8cbdf49f3daae2ac           1   

   shocks_detail__id v31102d  v31102a__0  v31102a__1  v31102a__2  v31102a__3  \
0                  0     Yes         NaN         NaN         NaN         NaN   
1                  0     Yes         NaN         NaN         NaN         NaN   
2                  0     Yes         NaN         NaN         NaN

In [10]:
shocks_df.head()

,Unnamed: 0,interview__key,interview__id,shocks__id,shocks_detail__id,v31102d,v31102a__0,v31102a__1,v31102a__2,v31102a__3,...,shockdate,corryear,v31127__0,v31127__1,v31127__2,v31127__3,v31127__4,v31127__5,v31127__98,specify_v31127
0,0,61-84-00-54,e4caefaf3c074773a6e478861d18f0a7,2,0,Yes,NaN,NaN,NaN,NaN,...,2022-01-01T00:00:00,2022.0,0,0,0,0,1,0,0,NaN
1,1,61-84-00-54,e4caefaf3c074773a6e478861d18f0a7,3,0,Yes,NaN,NaN,NaN,NaN,...,2020-01-01T00:00:00,2020.0,1,0,0,0,0,0,0,NaN
2,2,59-46-95-71,419de10d4606492390aaec2a7fb1e5b9,10,0,Yes,NaN,NaN,NaN,NaN,...,2020-08-01T00:00:00,2020.0,0,0,0,1,1,0,0,NaN
3,3,16-60-25-95,22b3b9365e3246b3821baad080732b6b,62,0,Yes,NaN,NaN,NaN,NaN,...,2021-07-01T00:00:00,2021.0,1,0,0,0,0,0,0,NaN
4,4,36-89-97-65,782b881d3751484d8cbdf49f3daae2ac,1,0,Yes,NaN,NaN,NaN,NaN,...,2021-04-01T00:00:00,2021.0,1,0,0,0,0,0,0,NaN


In [11]:
#Data cleaning and missing value replacement.

# Check for missing values
print("\nMissing values before cleaning:")
print(shocks_df.isnull().sum())

# Fill missing numerical values with 0
numeric_columns = ['shockdisinv', 'savusedshock', 'insusedshock', 'pubtransusedshock', 'borrusedshock']
shocks_df[numeric_columns] = shocks_df[numeric_columns].fillna(0)

# Verify that missing values are handled
print("\nMissing values after cleaning:")
print(shocks_df.isnull().sum())


Missing values before cleaning:
Unnamed: 0              0
interview__key          0
interview__id           0
shocks__id              0
shocks_detail__id       0
                     ... 
v31127__3               0
v31127__4               0
v31127__5               0
v31127__98              0
specify_v31127       2165
Length: 101, dtype: int64


KeyError: "None of [Index(['shockdisinv', 'savusedshock', 'insusedshock', 'pubtransusedshock',\n       'borrusedshock'],\n      dtype='object')] are in the [columns]"

In [ ]:
#Data exploration.

# Display basic statistics for numerical columns
print("\nBasic statistics for numerical columns:")
print(shocks_df[numeric_columns].describe())

# Calculate and display frequency of shock types
print("\nFrequency of shock types:")
shock_counts = shocks_df['typeshock'].value_counts()
print(shock_counts.head())

In [ ]:
# Create a figure with two subplots
plt.figure(figsize=(15, 6))

# Plot 1: Bar chart of shock types
plt.subplot(1, 2, 1)
plt.bar(range(10), shock_counts.head(10).values)
plt.xticks(range(10), shock_counts.head(10).index, rotation=45, ha='right')
plt.title('Top 10 Most Common Shock Types')
plt.ylabel('Frequency')

# Plot 2: Box plot of financial impacts
plt.subplot(1, 2, 2)
bp = plt.boxplot([shocks_df[col].dropna() for col in ['shockdisinv', 'savusedshock', 'insusedshock', 'pubtransusedshock']])
plt.xticks(range(1, 5), ['Disinvestment', 'Savings', 'Insurance', 'Public Transfer'], rotation=45)
plt.title('Distribution of Financial Impacts')
plt.ylabel('Amount')

plt.tight_layout()
plt.show()

In [ ]:
# Compare financial impact (shockdisinv) between two most common shock types
top_2_shocks = shock_counts.head(2).index
shock1_data = shocks_df[shocks_df['typeshock'] == top_2_shocks[0]]['shockdisinv']
shock2_data = shocks_df[shocks_df['typeshock'] == top_2_shocks[1]]['shockdisinv']

t_stat, p_val = stats.ttest_ind(shock1_data, shock2_data, nan_policy='omit')
print(f"\nT-test comparing financial impact between {top_2_shocks[0]} and {top_2_shocks[1]}:")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_val:.4f}")

In [ ]:
summary = {
    'Total Records': len(shocks_df),
    'Number of Unique Shock Types': shocks_df['typeshock'].nunique(),
    'Most Common Shock': shock_counts.index[0],
    'Average Financial Impact': shocks_df['shockdisinv'].mean(),
    'Total Financial Impact': shocks_df['shockdisinv'].sum()
}

for key, value in summary.items():
    print(f"{key}: {value}")

In [ ]:
print(shocksCol)

In [ ]:
shocks_df.head()

In [ ]:
shocksCol['typeshock'].hist(bins=20)
plt.title("Histogram of head_ethnicity")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# List of DataFrames to merge
dfs = [surveyV1_df, shocks_df, shocks_details_df]

# Merge all DataFrames on 'interview_key'
# Merge all DataFrames on 'interview_key'
merged_df = reduce(lambda left, right: pd.merge(left, right, on="interview_key", how="outer"), dfs)
#merged_df = reduce(lambda left, right: pd.merge(left, right, on="interview__key", how="outer"), dfs)
merged_df.fillna(0, inplace=True)

In [ ]:
mList = list(merged_df)
print(mList)

In [ ]:
# Select numerical columns for descriptive statistics
"""
shockRelated = [
'v31102__6', 'v31102__10', 'v31102__11', 'v31102__63', 'v31102__16', 'v31102__55', 'v31102__77', 'v31102__5', 'v31102__8', 'v31102__1',
 'v31102__2', 'v31102__3', 'v31102__24', 'v31102__46', 'v31102__62', 'v31102__18', 'v31102__21', 'v31102__22', 'v31102__70', 'v31102__90',
 'v31102b','v31105a', 'v31105b','v31106a','v31108a__1', 'v31108a__2', 'v31108a__3', 'v31108a__4', 'v31108a__5', 'v31108a__6', 'v31108a__7',
 'v31108__40', 'v31108__41', 'v31108__42', 'v31108__43', 'v31108__3', 'v31108__4', 'v31108__5', 'v31108__6', 'v31108__7', 'v31108__8', 'v31108__9', 
 'v31108__10', 'v31108__11', 'v31108__12', 'v31108__13', 'v31108__14', 'v31108__15', 'v31108__16', 'v31108__17', 'v31108__18', 'v31108__19', 
 'v31108__20', 'v31108__21', 'v31108__22', 'v31108__23', 'v31108__24', 'v31108__25', 'v31108__26', 'v31108__27', 'v31108__28', 'v31108__29', 
 'v31108__30', 'v31108__31', 'v31108__48', 'v31108__49', 'v31108__50', 'v31108__51', 'v31108__52', 'v31108__53', 'v31108__54', 'v31108__55', 
 'v31108__62', 'v31108__63', 'v31108__90', 'v31108__98']
 """

shockIndex_df = merged_df.iloc[:,"v31102__6"]
shockIndex_df.head()

In [ ]:
shockIndex_df = data.iloc[:,[shockRelated]]

In [ ]:

#shockRelated.head()

data.head()

# Compute descriptive statistics
#descriptive_stats = data[shockRelated].describe()
#print(descriptive_stats)

In [ ]:
# Select variables for correlation analysis
correlation_columns = ["PYIncCap", "hh_size", "head_age", "total_asset", "owned_land"]

# Compute correlation matrix
correlation_matrix = data[correlation_columns].corr()
print(correlation_matrix)

In [ ]:


# Heatmap of correlations
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

# Scatter plot: Income vs. Household Size
sns.scatterplot(x="hh_size", y="PYIncCap", data=data)
plt.title("Income vs. Household Size")
plt.xlabel("Household Size")
plt.ylabel("Per Capita Income")
plt.show()

In [ ]:
#Testing the corelation between no_shock and total_asset
# Select relevant columns
shocks_corelelation = ["no_shocks", "total_asset"]

# Drop rows with missing values in the selected columns
data_cleaned = data[shocks_corelelation].dropna()

# Descriptive statistics
print(data_cleaned.describe())

# Correlation analysis
correlation = data_cleaned["no_shocks"].corr(data_cleaned["total_asset"])
print(f"Correlation between no_shocks and total_asset: {correlation}")

# Scatter plot
plt.scatter(data_cleaned["no_shocks"], data_cleaned["total_asset"], alpha=0.5)
plt.title("Relationship between No. of Shocks and Total Assets")
plt.xlabel("Number of Shocks")
plt.ylabel("Total Assets")
plt.show()


In [ ]:
# Define the independent (X) and dependent (Y) variables
X = data_cleaned["no_shocks"]
Y = data_cleaned["total_asset"]

# Add a constant to the independent variable
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the regression summary
print(model.summary())